In [1]:
# pip install langchain --upgrade
# Version: 0.0.164

# !pip install pypdf

In [2]:
# PDF Loaders. If unstructured gives you a hard time, try PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv

load_dotenv()

True

### Load your data

In [3]:
loader = PyPDFLoader("../data/field-guide-to-data-science.pdf")

## Other options for loaders 
# loader = UnstructuredPDFLoader("../data/field-guide-to-data-science.pdf")
# loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")

In [4]:
data = loader.load()

In [10]:
# Note: If you're using PyPDFLoader then it will split by page for you already
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[5].page_content)} characters in your document')

You have 126 document(s) in your data
There are 2176 characters in your document


### Chunk your data up into smaller documents

In [6]:
# Note: If you're using PyPDFLoader then we'll be splitting for the 2nd time.
# This is optional, test out on your own data.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [7]:
print (f'Now you have {len(texts)} documents')

Now you have 162 documents


### Create embeddings of your documents to get ready for semantic search

In [12]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
from tqdm.autonotebook import tqdm

Check to see if there is an environment variable with you API keys, if not, use what you put below

In [13]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', 'YourAPIKey')

In [14]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

### Option #1: Pinecone
If you want to use pinecone, run the code below, if not then skip over to Chroma below it. You must go to [Pinecone.io](https://www.pinecone.io/) and set up an account

In [ ]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY', 'YourAPIKey')
PINECONE_API_ENV = os.getenv('PINECONE_API_ENV', 'us-east1-gcp') # You may need to switch with your env

# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchaintest" # put in the name of your pinecone index here

docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

### Option #2: Chroma

I like Chroma becauase it's local and easy to set up without an account

In [15]:
# load it into Chroma
docsearch = Chroma.from_documents(texts, embeddings)

In [18]:
query = "What is the data science process?"
docs = docsearch.similarity_search(query)

In [19]:
# Here's an example of the first document that was returned
print(docs[0].page_content[:450])

The Analytic  
Selection Process
If you focus only on the science aspect of Data Science you will 
never become a data artist.
A critical step in Data Science is to identify an analytic technique that 
will produce the desired action. Sometimes it is clear; a characteristic 
of the problem (e.g., data type) points to the technique you should 
implement. Other times, however, it can be difficult to know where 
to begin. The universe of possible an


### Query those docs to get your answer back

In [20]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [21]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [22]:
query = "What is the collect stage of data maturity?"
docs = docsearch.similarity_search(query)

In [23]:
chain.run(input_documents=docs, question=query)

' The collect stage of data maturity focuses on collecting internal or external datasets. Gathering sales records and corresponding weather data is an example of the collect stage.'